In [2]:
import os
import sys
spark_home = r"C:\spark\spark-4.0.0-bin-hadoop3"
sys.path.insert(0, spark_home + r"\python")
sys.path.insert(0, spark_home + r"\python\lib\py4j-0.10.9.9-src.zip")
os.environ["PYSPARK_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg,unix_timestamp, round as pyspark_round, count
from pyspark.sql.window import Window

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
students_df = spark.read.csv("H:/leet_code_sql_50/sql_50_leetcode/sample_data/12_students.csv", header=True, inferSchema=True)
print("Students DataFrame (from CSV):")
students_df.show()
students_df.printSchema() # Verify schema

subjects_df = spark.read.csv("H:/leet_code_sql_50/sql_50_leetcode/sample_data/12_subjects.csv", header=True, inferSchema=True)
print("Subjects DataFrame (from CSV):")
subjects_df.show()
subjects_df.printSchema() # Verify schema

examinations_df = spark.read.csv("H:/leet_code_sql_50/sql_50_leetcode/sample_data/12_examinations.csv", header=True, inferSchema=True)
print("Examinations DataFrame (from CSV):")
examinations_df.show()
examinations_df.printSchema() # Verify schema

all_combinations_df = students_df.crossJoin(subjects_df)
print("DataFrame after CROSS JOIN (all student-subject combinations):")
all_combinations_df.show()

joined_df = all_combinations_df.join(
    examinations_df,
    on=[
        all_combinations_df.student_id == examinations_df.student_id,
        all_combinations_df.subject_name == examinations_df.subject_name
    ],
    how="left_outer"
)

joined_df.show()

result_df = joined_df.groupBy(
    students_df.student_id,
    students_df.student_name,
    subjects_df.subject_name
).agg(
    count(examinations_df.student_id).alias("attended_exams")
)

result_df.show()

final_result_df = result_df.orderBy(
    students_df.student_id,
    subjects_df.subject_name
)

final_result_df.show()


Students DataFrame (from CSV):
+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|       Alice|
|         2|         Bob|
|         6|        Alex|
|        13|        John|
+----------+------------+

root
 |-- student_id: integer (nullable = true)
 |-- student_name: string (nullable = true)

Subjects DataFrame (from CSV):
+------------+
|subject_name|
+------------+
|        Math|
|     Physics|
| Programming|
+------------+

root
 |-- subject_name: string (nullable = true)

Examinations DataFrame (from CSV):
+----------+------------+
|student_id|subject_name|
+----------+------------+
|         1|        Math|
|         1|     Physics|
|         1| Programming|
|         2| Programming|
|         1|     Physics|
|         1|        Math|
|        13|        Math|
|        13| Programming|
|        13|     Physics|
|         2|        Math|
|         1|        Math|
+----------+------------+

root
 |-- student_id: integer (nullable = true)
 |-- s